## Install Advertools

In [1]:
%pip install advertools
%pip install pandas-gbq
%pip install Twisted==22.10.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.1/312.1 kB 3.9 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 277.2/277.2 kB 29.3 MB/s eta 0:00:00
  Using cached twython-3.9.1-py3-none-any.whl (33 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 55.4 MB/s eta 0:00:00:00:01
  Using cached cssselect-1.2.0-py2.py3-none-any.whl (18 kB)
  Using cached queuelib-1.6.2-py2.py3-none-any.whl (13 kB)
  Using cached service_identity-21.1.0-py2.py3-none-any.whl (12 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.5/241.5 kB 27.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.2/97.2 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 50.4 MB/s eta 0:00:0000:0100:01
  Using cached jmespath-1.0.1-py3-none-any.whl (20 kB)
  Using cached Automat-22.10.0-py2.py3-none-any.whl (26 kB)
  Using cached constantly-15.1.0-py2.py3-none-any.whl (7.9 kB)
  Using cached hyperlink-21.0.0-py2.py3-none-any.whl (74 kB)
  Using cached in

## Crawler

In [3]:
import advertools as adv
import pandas as pd
import pandas_gbq
from datetime import datetime

# Crawl URL // Create JL File
adv.crawl(
    'https://www.bergzeit.de/', 
    'output_file.jl', 
    # TODO !! validate this implementation if it works for your website
    xpath_selectors = {'robots_tag': '//meta[@name="robots"]/@content'},
    follow_links = True, 
    custom_settings = {
        'LOG_FILE': 'adv_crawl.log',
        'JOBDIR': 'crawl_job',
        'CONCURRENT_REQUESTS_PER_DOMAIN': 30,
        # Set this after consulting your infrastructure team, that the user agent is whitelisted
        'USER_AGENT':'custom_advertools_crawler'
        # set this attribute to test a small crawl
        # ,'CLOSESPIDER_PAGECOUNT': 100
    }
)

In [4]:
# Create Crawl Dataframe
crawl_df = pd.read_json('output_file.jl', lines=True)

# Set all column datatype to string 
crawl_df = crawl_df.astype(str)

# add crawl date
crawl_df.insert(0, 'date', datetime.strftime(datetime.now(), '%Y-%m-%d'))

# replace special characters in column names
crawl_df.columns = crawl_df.columns.str.replace(".", "_")
crawl_df.columns = crawl_df.columns.str.replace("-", "_")
crawl_df.columns = crawl_df.columns.str.replace("@", "")
crawl_df.columns = crawl_df.columns.str.replace(":", "_")

# Empty  values from high character count columns
high_char_columns = [
    'body_text', 'og_description', 
    'links_url', 'links_text', 'links_nofollow',
    'nav_links_url', 'nav_links_text', 'nav_links_nofollow',
    'header_links_url', 'header_links_text', 'header_links_nofollow',
    'jsonld_graph', 'jsonld_model', 'jsonld_review',
    'img_alt'
]
crawl_df = crawl_df.drop(columns=high_char_columns)

# Number of URLs + Dataframe
print(f'Gesamt: {crawl_df.shape}')
crawl_df.head(3)

Gesamt: (224, 84)


/opt/conda/bin/ipython:11: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  sys.exit(start_ipython())


,date,url,title,meta_desc,viewport,h1,h2,h3,canonical,alt_href,...,jsonld_aggregateRating_worstRating,og_image_width,og_image_height,og_image_type,twitter_label1,twitter_data1,twitter_label2,twitter_data2,img_loading,img_referrerpolicy
0,2022-12-08,https://www.bergzeit.de/,"Bergzeit Outdoor Shop - Bergsport, Klettern & ...",Finde hier alles rund um Bergsport ➤ Über 50.0...,"width=device-width, initial-scale=1.0, maximum...",\n Bergzeit Outdoor Shop - hier s...,\n Bergsport Online Shop mit prof...,Unsere Top Outdoor Kategorien@@Verantwortliche...,https://www.bergzeit.de/,https://www.bergzeit.de/@@https://www.bergzeit...,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1,2022-12-08,https://www.bergzeit.de/damen/bekleidung/hosen/,Hosen für Damen online kaufen | Bergzeit,Hosen für Damen im Bergzeit Online Shop ➤ Groß...,"width=device-width, initial-scale=1.0, maximum...",Hosen Damen (4170 Artikel),Datenschutzerklärung@@Social Media@@13. Versan...,Beliebte Kategorien@@Verantwortlicher für die ...,https://www.bergzeit.de/damen/bekleidung/hosen/,https://www.bergzeit.de/damen/bekleidung/hosen...,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
2,2022-12-08,https://www.bergzeit.de/damen/,Outdoor-Artikel für Damen online kaufen | Berg...,Damen Produkte im Bergzeit Online Shop ➤ Große...,"width=device-width, initial-scale=1.0, maximum...",Outdoor-Artikel für Damen (27283 Artikel),Datenschutzerklärung@@Social Media@@13. Versan...,Beliebte Kategorien@@Verantwortlicher für die ...,https://www.bergzeit.de/damen/,https://www.bergzeit.de/damen/@@https://www.be...,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


In [5]:
crawl_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 224 entries, 0 to 223
Data columns (total 84 columns):
 #   Column                                     Non-Null Count  Dtype 
---  ------                                     --------------  ----- 
 0   date                                       224 non-null    object
 1   url                                        224 non-null    object
 2   title                                      224 non-null    object
 3   meta_desc                                  224 non-null    object
 4   viewport                                   224 non-null    object
 5   h1                                         224 non-null    object
 6   h2                                         224 non-null    object
 7   h3                                         224 non-null    object
 8   canonical                                  224 non-null    object
 9   alt_href                                   224 non-null    object
 10  alt_hreflang                          

## Write to storage

In [ ]:
table_schema = [
    {'name':'date','type':'STRING'},
    {'name':'url','type':'STRING'},
    {'name':'title','type':'STRING'},
    {'name':'meta_desc','type':'STRING'},
    {'name':'viewport','type':'STRING'},
    {'name':'h1','type':'STRING'},
    {'name':'h2','type':'STRING'},
    {'name':'h3','type':'STRING'},
    {'name':'canonical','type':'STRING'},
    {'name':'alt_href','type':'STRING'},
    {'name':'alt_hreflang','type':'STRING'},
    {'name':'jsonld_context','type':'STRING'},
    {'name':'jsonld_type','type':'STRING'},
    {'name':'jsonld_url','type':'STRING'},
    {'name':'jsonld_potentialAction_type','type':'STRING'},
    {'name':'jsonld_potentialAction_query_input','type':'STRING'},
    {'name':'jsonld_potentialAction_target_urlTemplate','type':'STRING'},
    {'name':'jsonld_potentialAction_target_type','type':'STRING'},
    {'name':'jsonld_logo','type':'STRING'},
    {'name':'jsonld_name','type':'STRING'},
    {'name':'body_text','type':'STRING'},
    {'name':'size','type':'STRING'},
    {'name':'download_timeout','type':'STRING'},
    {'name':'download_slot','type':'STRING'},
    {'name':'download_latency','type':'STRING'},
    {'name':'depth','type':'STRING'},
    {'name':'status','type':'STRING'},
    {'name':'links_url','type':'STRING'},
    {'name':'links_text','type':'STRING'},
    {'name':'links_nofollow','type':'STRING'},
    {'name':'nav_links_url','type':'STRING'},
    {'name':'nav_links_text','type':'STRING'},
    {'name':'nav_links_nofollow','type':'STRING'},
    {'name':'header_links_url','type':'STRING'},
    {'name':'header_links_text','type':'STRING'},
    {'name':'header_links_nofollow','type':'STRING'},
    {'name':'ip_address','type':'STRING'},
    {'name':'crawl_time','type':'STRING'},
    {'name':'resp_headers_date','type':'STRING'},
    {'name':'resp_headers_content_type','type':'STRING'},
    {'name':'resp_headers_set_cookie','type':'STRING'},
    {'name':'resp_headers_x_frame_options','type':'STRING'},
    {'name':'resp_headers_x_powered_by','type':'STRING'},
    {'name':'resp_headers_x_content_type_options','type':'STRING'},
    {'name':'resp_headers_x_xss_protection','type':'STRING'},
    {'name':'resp_headers_cache_control','type':'STRING'},
    {'name':'resp_headers_pragma','type':'STRING'},
    {'name':'resp_headers_expires','type':'STRING'},
    {'name':'resp_headers_strict_transport_security','type':'STRING'},
    {'name':'resp_headers_vary','type':'STRING'},
    {'name':'resp_headers_content_language','type':'STRING'},
    {'name':'request_headers_accept','type':'STRING'},
    {'name':'request_headers_accept_language','type':'STRING'},
    {'name':'request_headers_user_agent','type':'STRING'},
    {'name':'request_headers_accept_encoding','type':'STRING'},
    {'name':'request_headers_cookie','type':'STRING'},
    {'name':'robots_tag','type':'STRING'},
    {'name':'jsonld_itemListElement','type':'STRING'},
    {'name':'jsonld_numberOfItems','type':'STRING'},
    {'name':'request_headers_referer','type':'STRING'},
    {'name':'charset','type':'STRING'},
    {'name':'og_locale','type':'STRING'},
    {'name':'og_type','type':'STRING'},
    {'name':'og_title','type':'STRING'},
    {'name':'og_description','type':'STRING'},
    {'name':'og_url','type':'STRING'},
    {'name':'og_site_name','type':'STRING'},
    {'name':'jsonld_graph','type':'STRING'},
    {'name':'img_width','type':'STRING'},
    {'name':'img_alt','type':'STRING'},
    {'name':'img_height','type':'STRING'},
    {'name':'img_src','type':'STRING'},
    {'name':'resp_headers_content_length','type':'STRING'},
    {'name':'resp_headers_age','type':'STRING'},
    {'name':'resp_headers_accept_ranges','type':'STRING'},
    {'name':'h4','type':'STRING'},
    {'name':'og_image','type':'STRING'},
    {'name':'og_image_width','type':'STRING'},
    {'name':'og_image_height','type':'STRING'},
    {'name':'og_image_type','type':'STRING'},
    {'name':'twitter_label1','type':'STRING'},
    {'name':'twitter_data1','type':'STRING'},
    {'name':'twitter_label2','type':'STRING'},
    {'name':'twitter_data2','type':'STRING'},
    {'name':'img_loading','type':'STRING'},
    {'name':'img_referrerpolicy','type':'STRING'},
    {'name':'jsonld_brand','type':'STRING'},
    {'name':'jsonld_category','type':'STRING'},
    {'name':'jsonld_description','type':'STRING'},
    {'name':'jsonld_image','type':'STRING'},
    {'name':'jsonld_model','type':'STRING'},
    {'name':'jsonld_offers','type':'STRING'},
    {'name':'jsonld_productId','type':'STRING'},
    {'name':'jsonld_review','type':'STRING'},
    {'name':'jsonld_aggregateRating_bestRating','type':'STRING'},
    {'name':'jsonld_aggregateRating_itemReviewed_type','type':'STRING'},
    {'name':'jsonld_aggregateRating_itemReviewed_name','type':'STRING'},
    {'name':'jsonld_aggregateRating_ratingValue','type':'STRING'},
    {'name':'jsonld_aggregateRating_reviewCount','type':'STRING'},
    {'name':'jsonld_aggregateRating_worstRating','type':'STRING'},
    {'name':'h5','type':'STRING'},
    {'name':'redirect_times','type':'STRING'},
    {'name':'redirect_ttl','type':'STRING'},
    {'name':'redirect_urls','type':'STRING'},
    {'name':'redirect_reasons','type':'STRING'},
    {'name':'footer_links_url','type':'STRING'},
    {'name':'footer_links_text','type':'STRING'},
    {'name':'footer_links_nofollow','type':'STRING'},
    {'name':'resp_headers_x_robots_tag','type':'STRING'},
    {'name':'retry_times','type':'STRING'},
    {'name':'resp_headers_etag','type':'STRING'},
    {'name':'resp_headers_last_modified','type':'STRING'},
    {'name':'resp_headers_x_request_id','type':'STRING'}
]

In [ ]:
# Limit the column names to the target schema
schema_column_names = [column['name'] for column in table_schema] 
df_columns_names = crawl_df.columns.tolist()

# Extract the subset of crawl columns that match the target schema
available_fields = [column for column in df_columns_names if column in schema_column_names] 
crawl_df_schema_checked = crawl_df[available_fields]

In [ ]:
# Define your target bigquery table here
project_id = 'bergzeit'
dataset_name = 'CrawlLogfileData'
table_name = 'advertools_crawl_logs'
full_table_name = dataset_name + '.' + table_name

def write_to_storage(df):
    # Write data to BigQuery
    pandas_gbq.to_gbq(df, full_table_name, project_id=project_id, table_schema=table_schema, if_exists='append')
    print('Feed uploaded')

write_to_storage(crawl_df_schema_checked)